In [2]:
import csv
import os
import random
import pickle
import itertools
import sklearn
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from nilearn import datasets
import nibabel as nib
from scipy.ndimage import center_of_mass
from scipy import stats
import matplotlib.pyplot as plt
import nibabel as nib
from nilearn import plotting
from IPython.display import display
from itertools import combinations_with_replacement

In [3]:
# Define modality renaming dictionary (modality_map)
modality_map = {
'hearing': 'Hearing',
'immune': 'Immune',
'renalhepatic': 'Renal & Hepatic',
'metabolic': 'Metabolic',
'cardiopulmonary': 'Cardiopulmonary',
'musculoskeletal': 'Musculoskeletal',
'bone_densitometry': 'Bone Densitometry of Heel',
'pwa': 'Pulse Wave Analysis',
'heart_mri': 'Heart MRI',
'carotid_ultrasound': 'Carotid Ultrasound',
'arterial_stiffness': 'Arterial Stiffness',
'ecg_rest': 'ECG at Rest',
'body_composition_by_impedance': 'Body Composition by Impedance',
'body_composition_dxa': 'Body Composition by DXA',
'bone_dxa': 'Bone Size, Mineral and Density by DXA',
'kidneys_mri': 'Kidney MRI',
'liver_mri': 'Liver MRI',
'abdominal_composition_mri_18_vars': 'Abdominal Composition by MRI',
'abdominal_organ_composition_mri_13_vars': 'Abdominal Organ Composition by MRI',
'struct_fast' : 'Regional grey matter volumes (FSL FAST)',
'struct_sub_first': 'Subcortical volumes (FSL FIRST)',

'struct_fs_aseg_mean_intensity' : 'ASEG Mean Intensity',
'struct_fs_aseg_volume' : 'ASEG Volume',


'struct_ba_exvivo_area' : 'BA ex-vivo Area',
'struct_ba_exvivo_mean_thickness' : 'BA ex-vivo Mean Thickness',
'struct_ba_exvivo_volume' : 'BA ex-vivo Volume',

'struct_a2009s_area' : 'a2009s Area',
'struct_a2009s_mean_thickness' : 'a2009s Mean Thickness',
'struct_a2009s_volume' : 'a2009s Volume',


'struct_dkt_area' : 'Desikan-Killiany-Tourville Area',
'struct_dkt_mean_thickness' : 'Desikan-Killiany-Tourville Mean Thickness',
'struct_dkt_volume' : 'Desikan-Killiany-Tourville Volume',


'struct_desikan_gw' : 'Desikan Grey/White Matter Contrast',
'struct_desikan_pial' : 'Desikan Pial',

'struct_desikan_white_area' : 'Desikan White Matter Area',
'struct_desikan_white_mean_thickness' : 'Desikan White Matter Mean Thickness',
'struct_desikan_white_volume' : 'Desikan White Matter Volume',
"struct_subsegmentation":'Subcortical Volumetric Subsegmentation',

'add_t1' : 'Whole-Brain T1w',
'add_t2' : 'Whole-Brain T2w',
"dwi_FA_tbss": "FA TBSS",
"dwi_FA_prob": "FA Probabilistic",
"dwi_MD_tbss": "MD TBSS",
"dwi_MD_prob": "MD Probabilistic",
"dwi_L1_tbss": "L1 TBSS",
"dwi_L1_prob": "L1 Probabilistic",
"dwi_L2_tbss": "L2 TBSS",
"dwi_L2_prob": "L2 Probabilistic",
"dwi_L3_tbss": "L3 TBSS",
"dwi_L3_prob": "L3 Probabilistic",
"dwi_MO_tbss": "MO TBSS",
"dwi_MO_prob": "MO Probabilistic",
"dwi_OD_tbss": "OD TBSS",
"dwi_OD_prob": "OD Probabilistic",
"dwi_ICVF_tbss": "ICVF TBSS",
"dwi_ICVF_prob": "ICVF Probabilistic",
"dwi_ISOVF_tbss": "ISOVF TBSS",
"dwi_ISOVF_prob": 'ISOVF Probabilistic',
"amplitudes_21": " 21 IC Amplitudes",
"amplitudes_55": "55 IC Amplitudes",
"full_correlation_21": "21 IC Full Correlation",
"full_correlation_55": "55 IC Full Correlation",
"partial_correlation_21": " 21 IC Partial Correlation",
"partial_correlation_55": " 55 IC Partial Correlation",
# aparc Tian S1 (I)
'aparc_Tian_S1_FA_i2': 'aparc-I FA',
'aparc_Tian_S1_Length_i2': 'aparc-I Length',
'aparc_Tian_S1_SIFT2_FBC_i2': 'aparc-I SIFT2 FBC',
'aparc_Tian_S1_Streamline_Count_i2': 'aparc-I Streamline Count',

# aparc a2009s Tian S1 (I)
'aparc_a2009s_Tian_S1_FA_i2': 'aparc.a2009s-I FA',
'aparc_a2009s_Tian_S1_Length_i2': 'aparc.a2009s-I Length',
'aparc_a2009s_Tian_S1_SIFT2_FBC_i2': 'aparc.a2009s-I SIFT2 FBC',
'aparc_a2009s_Tian_S1_Streamline_Count_i2': 'aparc.a2009s-I Streamline Count',

# Glasser Tian S1 (I)
'Glasser_Tian_S1_FA_i2': 'Glasser-I FA',
'Glasser_Tian_S1_Length_i2': 'Glasser-I Length',
'Glasser_Tian_S1_SIFT2_FBC_i2': 'Glasser-I SIFT2 FBC',
'Glasser_Tian_S1_Streamline_Count_i2': 'Glasser-I Streamline Count',

# Glasser Tian S4 (IV)
'Glasser_Tian_S4_FA_i2': 'Glasser-IV FA',
'Glasser_Tian_S4_Length_i2': 'Glasser-IV Length',
'Glasser_Tian_S4_SIFT2_FBC_i2': 'Glasser-IV SIFT2 FBC',
'Glasser_Tian_S4_Streamline_Count_i2': 'Glasser-IV Streamline Count',

# Schaefer7n1000p Tian S4 (IV) (in reality: Schaefer7n200p Tian S1)
'Schaefer7n1000p_Tian_S4_FA_i2': 'Schaefer7n200p-I FA', #'Schaefer7n1000p-IV FA',
'Schaefer7n1000p_Tian_S4_Length_i2': 'Schaefer7n200p-I Length',#'Schaefer7n1000p-IV Length',
'Schaefer7n1000p_Tian_S4_SIFT2_FBC_i2': 'Schaefer7n200p-I SIFT2 FBC',#'Schaefer7n1000p-IV SIFT2 FBC',
'Schaefer7n1000p_Tian_S4_Streamline_Count_i2': 'Schaefer7n200p-I Streamline Count', #'Schaefer7n1000p-IV Streamline Count'

# Schaefer7n200p Tian S4 (IV) (in reality: Schaefer7n500p Tian S4)
'Schaefer7n200p_Tian_S1_FA_i2': 'Schaefer7n500p-IV FA',
'Schaefer7n200p_Tian_S1_Length_i2': 'Schaefer7n500p-IV Length',
'Schaefer7n200p_Tian_S1_SIFT2_FBC_i2': 'Schaefer7n500p-IV SIFT2 FBC',
'Schaefer7n200p_Tian_S1_Streamline_Count_i2': 'Schaefer7n500p-IV Streamline Count',

# Schaefer7n500p Tian S4 (IV) (in reality: Schaefer7n1000p Tian S4)
'Schaefer7n500p_Tian_S4_FA_i2': 'Schaefer7n1000p-IV FA',
'Schaefer7n500p_Tian_S4_Length_i2': 'Schaefer7n1000p-IV Length',
'Schaefer7n500p_Tian_S4_SIFT2_FBC_i2': 'Schaefer7n1000p-IV SIFT2 FBC',
'Schaefer7n500p_Tian_S4_Streamline_Count_i2': 'Schaefer7n1000p-IV Streamline Count',

# Resting state 
'full_correlation_aparc_a2009s_Tian_S1' : 'aparc.a2009s-I Full Correlation',
'full_correlation_aparc_Tian_S1': 'aparc-I Full Correlation',
'full_correlation_Glasser_Tian_S1': 'Glasser-I Full Correlation',
'full_correlation_Glasser_Tian_S4': 'Glasser-IV Full Correlation',
'full_correlation_Schaefer7n200p_Tian_S1': 'Schaefer7n200p-I Full Correlation',
'full_correlation_Schaefer7n500p_Tian_S4': 'Schaefer7n500p-IV Full Correlation',
'partial_correlation_aparc_a2009s_Tian_S1': 'aparc.a2009s-I Partial Correlation',
'partial_correlation_aparc_Tian_S1': 'aparc-I Partial Correlation',
'partial_correlation_Glasser_Tian_S1': 'Glasser-I Partial Correlation',
'partial_correlation_Glasser_Tian_S4': 'Glasser-IV Partial Correlation',
'partial_correlation_Schaefer7n200p_Tian_S1': 'Schaefer7n200p-I Partial Correlation',
'partial_correlation_Schaefer7n500p_Tian_S4': 'Schaefer7n500p-IV Partial Correlation',

'lifestyle-envir': 'Lifestyle & Environment',

'allmri': '3 Brain MRI Modalities Stacked',
'dwi': 'Brain dwMRI Stacked',
'smri': 'Brain sMRI Stacked',
'rs': 'Brain rsMRI Stacked',
'body': 'Body Physiology Stacked',
'body-comp': 'Body Composition Stacked',
#'cardiopulmonary': 'Cardiopulmonary stacked',
'renal-hepatic': 'Renal & Hepatic Stacked',
'lifestyle-envir': 'Lifestyle & Environment',
'brain-plus-body': '3 Brain MRI Modalities & Body Stacked',
'brain-body': 'Brain & Body Stacked',
'body-only': 'Body Physiology and Composition Stacked' 
}

# Define modality names for renaming
modality_names = {
'hearing': 'Hearing',
'immune': 'Immune',
'renalhepatic': 'Renal & Hepatic',
'metabolic': 'Metabolic',
'cardiopulmonary': 'Cardiopulmonary',
'musculoskeletal': 'Musculoskeletal',
'bone_densitometry': 'Bone Densitometry of Heel',
'pwa': 'Pulse Wave Analysis',
'heart_mri': 'Heart MRI',
'carotid_ultrasound': 'Carotid Ultrasound',
'arterial_stiffness': 'Arterial Stiffness',
'ecg_rest': 'ECG at Rest',
'body_composition_by_impedance': 'Body Composition by Impedance',
'body_composition_dxa': 'Body Composition by DXA',
'bone_dxa': 'Bone Size, Mineral and Density by DXA',
'kidneys_mri': 'Kidney MRI',
'liver_mri': 'Liver MRI',
'abdominal_composition_mri_18_vars': 'Abdominal Composition by MRI',
'abdominal_organ_composition_mri_13_vars': 'Abdominal Organ Composition by MRI',
'struct_fast' : 'Regional grey matter volumes (FSL FAST)',
'struct_sub_first': 'Subcortical volumes (FSL FIRST)',

'struct_fs_aseg_mean_intensity' : 'ASEG Mean Intensity',
'struct_fs_aseg_volume' : 'ASEG Volume',


'struct_ba_exvivo_area' : 'BA ex-vivo Area',
'struct_ba_exvivo_mean_thickness' : 'BA ex-vivo Mean Thickness',
'struct_ba_exvivo_volume' : 'BA ex-vivo Volume',

'struct_a2009s_area' : 'a2009s Area',
'struct_a2009s_mean_thickness' : 'a2009s Mean Thickness',
'struct_a2009s_volume' : 'a2009s Volume',


'struct_dkt_area' : 'Desikan-Killiany-Tourville Area',
'struct_dkt_mean_thickness' : 'Desikan-Killiany-Tourville Mean Thickness',
'struct_dkt_volume' : 'Desikan-Killiany-Tourville Volume',


'struct_desikan_gw' : 'Desikan Grey/White Matter Contrast',
'struct_desikan_pial' : 'Desikan Pial',

'struct_desikan_white_area' : 'Desikan White Matter Area',
'struct_desikan_white_mean_thickness' : 'Desikan White Matter Mean Thickness',
'struct_desikan_white_volume' : 'Desikan White Matter Volume',
"struct_subsegmentation":'Subcortical Volumetric Subsegmentation',

'add_t1' : 'Whole-Brain T1w',
'add_t2' : 'Whole-Brain T2w',
"dwi_FA_tbss": "FA TBSS",
"dwi_FA_prob": "FA Probabilistic",
"dwi_MD_tbss": "MD TBSS",
"dwi_MD_prob": "MD Probabilistic",
"dwi_L1_tbss": "L1 TBSS",
"dwi_L1_prob": "L1 Probabilistic",
"dwi_L2_tbss": "L2 TBSS",
"dwi_L2_prob": "L2 Probabilistic",
"dwi_L3_tbss": "L3 TBSS",
"dwi_L3_prob": "L3 Probabilistic",
"dwi_MO_tbss": "MO TBSS",
"dwi_MO_prob": "MO Probabilistic",
"dwi_OD_tbss": "OD TBSS",
"dwi_OD_prob": "OD Probabilistic",
"dwi_ICVF_tbss": "ICVF TBSS",
"dwi_ICVF_prob": "ICVF Probabilistic",
"dwi_ISOVF_tbss": "ISOVF TBSS",
"dwi_ISOVF_prob": 'ISOVF Probabilistic',
"amplitudes_21": " 21 IC Amplitudes",
"amplitudes_55": "55 IC Amplitudes",
"full_correlation_21": "21 IC Full Correlation",
"full_correlation_55": "55 IC Full Correlation",
"partial_correlation_21": " 21 IC Partial Correlation",
"partial_correlation_55": " 55 IC Partial Correlation",
# aparc Tian S1 (I)
'aparc_Tian_S1_FA_i2': 'aparc-I FA',
'aparc_Tian_S1_Length_i2': 'aparc-I Length',
'aparc_Tian_S1_SIFT2_FBC_i2': 'aparc-I SIFT2 FBC',
'aparc_Tian_S1_Streamline_Count_i2': 'aparc-I Streamline Count',

# aparc a2009s Tian S1 (I)
'aparc_a2009s_Tian_S1_FA_i2': 'aparc.a2009s-I FA',
'aparc_a2009s_Tian_S1_Length_i2': 'aparc.a2009s-I Length',
'aparc_a2009s_Tian_S1_SIFT2_FBC_i2': 'aparc.a2009s-I SIFT2 FBC',
'aparc_a2009s_Tian_S1_Streamline_Count_i2': 'aparc.a2009s-I Streamline Count',

# Glasser Tian S1 (I)
'Glasser_Tian_S1_FA_i2': 'Glasser-I FA',
'Glasser_Tian_S1_Length_i2': 'Glasser-I Length',
'Glasser_Tian_S1_SIFT2_FBC_i2': 'Glasser-I SIFT2 FBC',
'Glasser_Tian_S1_Streamline_Count_i2': 'Glasser-I Streamline Count',

# Glasser Tian S4 (IV)
'Glasser_Tian_S4_FA_i2': 'Glasser-IV FA',
'Glasser_Tian_S4_Length_i2': 'Glasser-IV Length',
'Glasser_Tian_S4_SIFT2_FBC_i2': 'Glasser-IV SIFT2 FBC',
'Glasser_Tian_S4_Streamline_Count_i2': 'Glasser-IV Streamline Count',

# Schaefer7n1000p Tian S4 (IV) (in reality: Schaefer7n200p Tian S1)
'Schaefer7n1000p_Tian_S4_FA_i2': 'Schaefer7n200p-I FA', #'Schaefer7n1000p-IV FA',
'Schaefer7n1000p_Tian_S4_Length_i2': 'Schaefer7n200p-I Length',#'Schaefer7n1000p-IV Length',
'Schaefer7n1000p_Tian_S4_SIFT2_FBC_i2': 'Schaefer7n200p-I SIFT2 FBC',#'Schaefer7n1000p-IV SIFT2 FBC',
'Schaefer7n1000p_Tian_S4_Streamline_Count_i2': 'Schaefer7n200p-I Streamline Count', #'Schaefer7n1000p-IV Streamline Count'

# Schaefer7n200p Tian S4 (IV) (in reality: Schaefer7n500p Tian S4)
'Schaefer7n200p_Tian_S1_FA_i2': 'Schaefer7n500p-IV FA',
'Schaefer7n200p_Tian_S1_Length_i2': 'Schaefer7n500p-IV Length',
'Schaefer7n200p_Tian_S1_SIFT2_FBC_i2': 'Schaefer7n500p-IV SIFT2 FBC',
'Schaefer7n200p_Tian_S1_Streamline_Count_i2': 'Schaefer7n500p-IV Streamline Count',

# Schaefer7n500p Tian S4 (IV) (in reality: Schaefer7n1000p Tian S4)
'Schaefer7n500p_Tian_S4_FA_i2': 'Schaefer7n1000p-IV FA',
'Schaefer7n500p_Tian_S4_Length_i2': 'Schaefer7n1000p-IV Length',
'Schaefer7n500p_Tian_S4_SIFT2_FBC_i2': 'Schaefer7n1000p-IV SIFT2 FBC',
'Schaefer7n500p_Tian_S4_Streamline_Count_i2': 'Schaefer7n1000p-IV Streamline Count',

# Resting state 
'full_correlation_aparc_a2009s_Tian_S1' : 'aparc.a2009s-I Full Correlation',
'full_correlation_aparc_Tian_S1': 'aparc-I Full Correlation',
'full_correlation_Glasser_Tian_S1': 'Glasser-I Full Correlation',
'full_correlation_Glasser_Tian_S4': 'Glasser-IV Full Correlation',
'full_correlation_Schaefer7n200p_Tian_S1': 'Schaefer7n200p-I Full Correlation',
'full_correlation_Schaefer7n500p_Tian_S4': 'Schaefer7n500p-IV Full Correlation',
'partial_correlation_aparc_a2009s_Tian_S1': 'aparc.a2009s-I Partial Correlation',
'partial_correlation_aparc_Tian_S1': 'aparc-I Partial Correlation',
'partial_correlation_Glasser_Tian_S1': 'Glasser-I Partial Correlation',
'partial_correlation_Glasser_Tian_S4': 'Glasser-IV Partial Correlation',
'partial_correlation_Schaefer7n200p_Tian_S1': 'Schaefer7n200p-I Partial Correlation',
'partial_correlation_Schaefer7n500p_Tian_S4': 'Schaefer7n500p-IV Partial Correlation',

'lifestyle-envir': 'Lifestyle & Environment',

'allmri': '3 Brain MRI Modalities Stacked',
'dwi': 'Brain dwMRI Stacked',
'smri': 'Brain sMRI Stacked',
'rs': 'Brain rsMRI Stacked',
'body': 'Body Physiology Stacked',
'body-comp': 'Body Composition Stacked',
#'cardiopulmonary': 'Cardiopulmonary stacked',
'renal-hepatic': 'Renal & Hepatic Stacked',
'lifestyle-envir': 'Lifestyle & Environment',
'brain-plus-body': '3 Brain MRI Modalities & Body Stacked',
'brain-body': 'Brain & Body Stacked',
'body-only': 'Body Physiology and Composition Stacked' 
}

# Correlation between neuroimaging features and composite brain marker

In [ ]:
# Configuration
warnings.simplefilter(action='ignore', category=FutureWarning)
from datetime import datetime

folds = range(0, 5)
base_path = '/UK_BB/brainbody'
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
modalities_brain = [
    'Schaefer7n200p_Tian_S1_Streamline_Count_i2', # this corresponds to 500pS4 atlas with 554 structures
    'full_correlation_Schaefer7n500p_Tian_S4', # this corresponds to 500pS4 atlas with 554 structures
    'struct_fs_aseg_volume'
]

Compute correlations with composite brain

In [ ]:
# Get correlations
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Compute correlation between features and predicted g-factor for brain modalities
modality_results = {}
unsorted_results = {}

# Process each brain modality
for modality in modalities_brain:
    print(f"\nProcessing modality: {modality}")
    features, g_pred = [], []
    
    for fold in folds:
        try:
            # Path to predicted g-factor values
            test_path = os.path.join(
                base_path,
                'stacking',
                'brain',
                'allmri',
                'folds', 
                f'fold_{fold}',
                'g_pred',
                f'allmri_target_pred_2nd_level_rf_test_fold_{fold}.csv'
            )
            
            # Path to feature values
            features_path = os.path.join(
                base_path,
                'brain',
                'folds',
                f'fold_{fold}',
                'scaling',
                f'{modality}_test_deconf_fold_{fold}.csv'
            )

            features_eid_path = os.path.join(
                base_path,
                'brain',
                'folds',
                f'fold_{fold}',
                'g_pred',
                f'{modality}_g_pred_XGB_test_with_id_fold_{fold}.csv'
            )

            # Read data
            g_pred_test = pd.read_csv(test_path)
            features_corrected = pd.read_csv(features_path)
            features_eid = pd.read_csv(features_eid_path)['eid']
            features_eid = pd.DataFrame(features_eid, columns = ['eid'])

            # Concat
            features_with_eid = pd.concat([features_corrected, features_eid], axis=1)
            
            # Merge on eid
            merged_df = pd.merge(
                features_with_eid,
                g_pred_test,
                on='eid',
                how='inner'
            )
            
            # Extract aligned data (remove eid column)
            features_aligned = merged_df.drop(columns=['eid', f'g_pred_stack_test'])
            g_aligned = merged_df[[f'g_pred_stack_test']]
            g_aligned = g_aligned.rename(columns={f'g_pred_stack_test': f'g_pred_test_{modality}'})
            
            # Append aligned data
            features.append(features_aligned)
            g_pred.append(g_aligned)
            
        except Exception as e:
            print(f"Error processing fold {fold} for {modality}: {str(e)}")
            continue
    
    # Check if any data was collected
    if len(features) == 0 or len(g_pred) == 0:
        print(f"No data collected for {modality}, skipping...")
        continue
    
    # Concatenate across folds (now data is properly aligned within each fold)
    features_concat = pd.concat(features, axis=0, ignore_index=True)
    g_pred_concat = pd.concat(g_pred, axis=0, ignore_index=True)


    correlations = []
    for feature in features_concat.columns:
        try:
            # Get data
            x = g_pred_concat[f'g_pred_test_{modality}'].values
            y = features_concat[feature].values
            
            # Create mask for non-NaN values
            mask = ~np.isnan(x) & ~np.isnan(y)
            valid_count = sum(mask)
            
            # Debug print for problematic features
            if valid_count < 10:
                print(f"Low valid pairs ({valid_count}) for feature: {feature}")
            
            # Skip if insufficient data
            if valid_count < 2:
                print(f"Skipping {feature} - insufficient data (n={valid_count})")
                correlations.append({
                    'Feature': feature,
                    'Pearson r': np.nan,
                    'p-value': np.nan,
                    'Valid_pairs': valid_count
                })
                continue
            
            # Check for zero variance
            if np.std(x[mask]) == 0 or np.std(y[mask]) == 0:
                print(f"Skipping {feature} - zero variance in x or y")
                correlations.append({
                    'Feature': feature,
                    'Pearson r': np.nan,
                    'p-value': np.nan,
                    'Valid_pairs': valid_count
                })
                continue
            
            # Calculate correlation
            r_pred, p_pred = stats.pearsonr(x[mask], y[mask])
            
            correlations.append({
                'Feature': feature,
                'Pearson r': r_pred,
                'p-value': p_pred,
                'Valid_pairs': valid_count
            })
            
        except Exception as e:
            print(f"Error calculating {feature} in {modality}: {str(e)}")
            correlations.append({
                'Feature': feature,
                'Pearson r': np.nan,
                'p-value': np.nan,
                'Valid_pairs': np.nan
            })
    
    # Create DataFrame
    df = pd.DataFrame(correlations)
    df['Modality'] = modality
    
    # Store results
    unsorted_results[modality] = df.copy()
    modality_results[modality] = df.sort_values('Pearson r', ascending=False, key=abs).reset_index(drop=True)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.join(base_path, 'feature_imp', 'feature_imp_brain'), exist_ok=True)
    
    # Save individual CSV
    output_path = os.path.join(base_path, f'feature_imp/feature_imp_brain/{modality}_corr_with_g_stack.csv')
    df.round(4).to_csv(output_path, index=False)
    print(f"Saved CSV for {modality} with {len(df)} features")

# Save Excel files for brain modalities
try:
    with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'brain_modalities_unsorted_stack_{timestamp}.xlsx'), 
                       engine='openpyxl') as writer:
        for modality, df in unsorted_results.items():
            try:
                sheet_name = modality[:31]
                df.round(4).to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error saving {modality} to unsorted Excel: {str(e)}")
                continue

    with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'brain_modalities_sorted_stack_full.xlsx'),
                       engine='openpyxl') as writer:
        
        # Individual sorted sheets
        for modality, df in modality_results.items():
            try:
                sheet_name = modality[:31]
                df.round(4).to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error saving {modality} to sorted Excel: {str(e)}")
                continue
        
        # Combined sorted sheet
        combined_sorted = pd.concat(
                [df.assign(Modality_Pretty=modality_names.get(modality, modality)) 
                 for modality, df in modality_results.items()],
                axis=0, ignore_index=True)
        combined_sorted.round(4).to_excel(writer, sheet_name='All_brain_sorted_stack_', index=False)

except Exception as e:
    print(f"Error creating Excel files: {str(e)}")

print("\nProcessing complete! Created:")
print(f"- Individual CSV files for each brain modality in 'feature_imp_brain' folder")
print(f"- brain_modalities_unsorted_stack_{timestamp}.xlsx")
print(f"- brain_modalities_sorted_stack_{timestamp}.xlsx (with 'All_brain_sorted_stack_' sheet)")

Check shapes

For a connectome with 554 regions:

- Total possible connections: 554 × 554 = 306,916
- Without diagonal (self-connections): 306,916 - 554 = 306,362
- Upper triangle only (including diagonal): 554 × 555 ÷ 2 = 153,735
- Upper triangle only (excluding diagonal): 553 × 554 ÷ 2 = 153,181

1. Diffusion MRI Approach (153,735 pairs)
Includes diagonal elements (self-connections)

*Uses combinations_with_replacement which includes pairs where i=j*

- Formula: n×(n+1)/2 = 554×555/2 = 153,735

2. Functional MRI Approach (153,181 pairs)
Excludes diagonal elements (no self-connections)

*Uses standard combinations (i<j) which excludes pairs where i=j*

- Formula: n×(n-1)/2 = 554×553/2 = 153,181

In [ ]:
# Get shape of the original matrices: dwMRI
with open('/UK_BB/brainbody/brain/folds/fold_0/scaling/Schaefer7n200p_Tian_S1_FA_i2_test_deconf_fold_0.csv', 'r') as f:
    reader = csv.reader(f)
    row_count = sum(1 for row in reader)
    
with open('/UK_BB/brainbody/brain/folds/fold_0/scaling/Schaefer7n200p_Tian_S1_FA_i2_test_deconf_fold_0.csv', 'r') as f:
    col_count = len(next(csv.reader(f)))
    
print(f"Shape dwMRI: ({row_count}, {col_count})")

Shape dwMRI: (5171, 153735)


In [ ]:
# Compute flattened array shape: dwMRI
n = 554

# 1. Total flattened
total_flattened = n * n  # = 306,916
print(f"Full 554x554 flattened: {total_flattened} elements")  # 306,916

# 2. Upper triangle including diagonal
upper_triangle_with_diag = n * (n + 1) // 2  # = 153,735
print(f"Upper triangle with diagonal: {upper_triangle_with_diag} elements")  # 153,735

# 3. Verify
arr = np.random.rand(n, n)

# Get upper triangle indices (including diagonal)
indices = np.triu_indices(n)  # k=0 includes diagonal
vectorized = arr[indices]
print(f"Actual vectorized shape: {vectorized.shape}")  # (153735,)

Full 554x554 flattened: 306916 elements
Upper triangle with diagonal: 153735 elements
Actual vectorized shape: (153735,)


In [ ]:
# Get shape of the original matrices
with open('/UK_BB/brainbody/brain/folds/fold_0/scaling/full_correlation_Schaefer7n500p_Tian_S4_test_deconf_fold_0.csv', 'r') as f:
    reader = csv.reader(f)
    row_count = sum(1 for row in reader)
    
with open('/UK_BB/brainbody/brain/folds/fold_0/scaling/full_correlation_Schaefer7n500p_Tian_S4_test_deconf_fold_0.csv', 'r') as f:
    col_count = len(next(csv.reader(f)))
    
print(f"Shape rsMRI: ({row_count}, {col_count})")

Shape rsMRI: (5099, 153181)


In [19]:
# Compute flattened array shape: rsMRI
n = 554

# 1. Upper triangle excluding diagonal
upper_triangle_no_diag = n * (n - 1) // 2  # = 153,181
print(f"Upper triangle without diagonal: {upper_triangle_no_diag} elements")  # 153,181

# 2. Verify with actual array
arr = np.random.rand(n, n)

# Get upper triangle indices excluding diagonal (k=1 starts above diagonal)
indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
vectorized = arr[indices]
print(f"Actual vectorized shape (no diagonal): {vectorized.shape}")  # (153181,)

Upper triangle without diagonal: 153181 elements
Actual vectorized shape (no diagonal): (153181,)


# dwMRI and rsMRI

In [10]:
# dwMRI
corr_dwi = pd.read_csv( os.path.join(base_path, f'feature_imp/feature_imp_brain/Schaefer7n200p_Tian_S1_Streamline_Count_i2_corr_with_g_stack.csv')) # 1D array of 153,735 correlations
print(corr_dwi.shape)

(153735, 5)


In [12]:
# rsMRI
corr_rs = pd.read_csv(os.path.join(base_path, f'feature_imp/feature_imp_brain/full_correlation_Schaefer7n500p_Tian_S4_corr_with_g_stack.csv')) # 1D array of 153,735 correlations
print(corr_rs.shape)

(153181, 5)


### Map correlations to connectomes

1. Get region labels

In [ ]:
# Cortical (Schaefer 500x7 Atlas)
schaefer = datasets.fetch_atlas_schaefer_2018(n_rois=500, yeo_networks=7)
cortical_labels = [label.decode('utf-8') for label in schaefer.labels]

# First 10 labels from fetch_atlas_schaefer_2018
print("Labels from atlas:", cortical_labels[:10])

# Subcortical (Tian S4 3T)
tian_label_path = "/UK_BB/brainbody/feature_imp/feature_imp_brain/Tian2020MSA/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S4_3T_label.txt"
with open(tian_label_path, 'r') as f:
    subcort_labels = [line.strip() for line in f if line.strip()]  # Remove empty lines

# Combined labels
all_labels = cortical_labels + subcort_labels
n_total = len(all_labels)   # 500 (cortical) + 54 (subcortical)

print(f"Total regions: {n_total}")
print(len(subcort_labels))
print("Cortical:", cortical_labels)
print("Subcortical:", subcort_labels)


[get_dataset_dir] Dataset found in /home/irinabu/nilearn_data/schaefer_2018

Labels from atlas: ['7Networks_LH_Vis_1', '7Networks_LH_Vis_2', '7Networks_LH_Vis_3', '7Networks_LH_Vis_4', '7Networks_LH_Vis_5', '7Networks_LH_Vis_6', '7Networks_LH_Vis_7', '7Networks_LH_Vis_8', '7Networks_LH_Vis_9', '7Networks_LH_Vis_10']
Total regions: 554
54
Cortical: ['7Networks_LH_Vis_1', '7Networks_LH_Vis_2', '7Networks_LH_Vis_3', '7Networks_LH_Vis_4', '7Networks_LH_Vis_5', '7Networks_LH_Vis_6', '7Networks_LH_Vis_7', '7Networks_LH_Vis_8', '7Networks_LH_Vis_9', '7Networks_LH_Vis_10', '7Networks_LH_Vis_11', '7Networks_LH_Vis_12', '7Networks_LH_Vis_13', '7Networks_LH_Vis_14', '7Networks_LH_Vis_15', '7Networks_LH_Vis_16', '7Networks_LH_Vis_17', '7Networks_LH_Vis_18', '7Networks_LH_Vis_19', '7Networks_LH_Vis_20', '7Networks_LH_Vis_21', '7Networks_LH_Vis_22', '7Networks_LH_Vis_23', '7Networks_LH_Vis_24', '7Networks_LH_Vis_25', '7Networks_LH_Vis_26', '7Networks_LH_Vis_27', '7Networks_LH_Vis_28', '7Networks_LH_Vis_29', '7Networks_LH_Vis_30', '7Networks_LH_Vis_31', '7Networks_LH_Vis_32',

2. Get MNI coordinates

In [ ]:
# Subcortical (Tian S4 3T)
tian_img = nib.load("/UK_BB/brainbody/feature_imp/feature_imp_brain/Tian2020MSA/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S4_3T.nii.gz") 
subcort_labels = np.loadtxt("/UK_BB/brainbody/feature_imp/feature_imp_brain/Tian2020MSA/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S4_3T_label.txt", dtype=str)  # Region names
tian_cog = np.loadtxt("/UK_BB/brainbody/feature_imp/feature_imp_brain/Tian2020MSA/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S4_3T_COG.txt")  # Pre-computed MNI centroids (mm) - Center of Gravity

# Use pre-computed MNI coordinates (more accurate than centroid)
mni_coords_subcortical = tian_cog[:, :3]  # Columns 1-3 are MNI coordinates
print(f"MSA IV - Atlas data shape: {subcort_labels.shape}")  # Should be (54,)
print(f"MSA IV - Number of ROIs: {len(subcort_labels)}")
print(f"MSA IV - Number of coordinates calculated: {len(mni_coords_subcortical)}")
print(f"MSA IV - MNI coordinates shape: {mni_coords_subcortical.shape}")  # Should be (54, 3)
print("MSA IV - First 5 coordinates:")
print(mni_coords_subcortical[:5])

print("Tian labels:", subcort_labels[:10])
print("Tian coords:", mni_coords_subcortical[:10])
print("Counts:", len(subcort_labels), len(mni_coords_subcortical))

MSA IV - Atlas data shape: (54,)
MSA IV - Number of ROIs: 54
MSA IV - Number of coordinates calculated: 54
MSA IV - MNI coordinates shape: (54, 3)
MSA IV - First 5 coordinates:
[[ 20. -12. -22.]
 [ 22. -18. -16.]
 [  8. -14.   4.]
 [  6.  -6.   2.]
 [ 30. -14. -20.]]
Tian labels: ['HIP-head-m1-rh' 'HIP-head-m2-rh' 'THA-VAip-rh' 'THA-VAia-rh'
 'HIP-head-l-rh' 'HIP-body-rh' 'HIP-tail-rh' 'THA-VPm-rh' 'THA-VPl-rh'
 'THA-VAs-rh']
Tian coords: [[ 20. -12. -22.]
 [ 22. -18. -16.]
 [  8. -14.   4.]
 [  6.  -6.   2.]
 [ 30. -14. -20.]
 [ 30. -28. -10.]
 [ 24. -38.  -2.]
 [ 10. -24.   2.]
 [ 18. -22.   4.]
 [ 10. -10.  12.]]
Counts: 54 54


In [40]:
# Cortical (Schaefer 500x7 Atlas)
# Fetch Schaefer atlas (500 ROIs, 7 networks, 2mm resolution)
schaefer_atlas = datasets.fetch_atlas_schaefer_2018(
    n_rois=500,
    yeo_networks=7,
    resolution_mm=2,
    data_dir=None,
    verbose=1
)

# Load the atlas
schaefer_img = nib.load(schaefer_atlas.maps)
schaefer_data = schaefer_img.get_fdata()

# Unique ROI numbers from the NIfTI
roi_numbers_from_img = np.unique(schaefer_data)[1:]  # skip background

for i in range(10):
    print(f"ROI {roi_numbers_from_img[i]} -> Label {cortical_labels[i]}")
    
# First 10 ROI numbers
print("ROI numbers from image:", roi_numbers_from_img[:10])
print(f"Schaefer - Atlas data shape: {schaefer_data.shape}")

[get_dataset_dir] Dataset found in /home/irinabu/nilearn_data/schaefer_2018

ROI 1.0 -> Label 7Networks_LH_Vis_1
ROI 2.0 -> Label 7Networks_LH_Vis_2
ROI 3.0 -> Label 7Networks_LH_Vis_3
ROI 4.0 -> Label 7Networks_LH_Vis_4
ROI 5.0 -> Label 7Networks_LH_Vis_5
ROI 6.0 -> Label 7Networks_LH_Vis_6
ROI 7.0 -> Label 7Networks_LH_Vis_7
ROI 8.0 -> Label 7Networks_LH_Vis_8
ROI 9.0 -> Label 7Networks_LH_Vis_9
ROI 10.0 -> Label 7Networks_LH_Vis_10
ROI numbers from image: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Schaefer - Atlas data shape: (91, 109, 91)


In [35]:
# Get cotical labels except background (0)
labels = np.unique(schaefer_data)[1:]  
print(f"Schaefer - Number of ROIs: {len(labels)}")  # 500

# Compute centroids using center_of_mass
mni_coords_cortical = []
for label in labels:
    # Create binary mask for the current ROI
    roi_mask = (schaefer_data == label)
    
    # Calculate center of mass in voxel space
    try:
        ijk = center_of_mass(roi_mask)
        # Convert to MNI space
        xyz = nib.affines.apply_affine(schaefer_img.affine, ijk)
        mni_coords_cortical.append(xyz)
    except Exception as e:
        print(f"Error processing label {label}: {str(e)}")
        continue

mni_coords_cortical = np.array(mni_coords_cortical)  # Shape: (500, 3)
print("Centroid count:", len(mni_coords_cortical))
print("Label count:", len(cortical_labels))
# Verify
print(f"Schaefer - Number of coordinates: {len(mni_coords_cortical)}")
print(f"Schaefer - MNI coordinates shape: {mni_coords_cortical.shape}")  # Should be (500, 3)
print("Schaefer - First 5 coordinates:")
print(mni_coords_cortical[:5])

Schaefer - Number of ROIs: 500
Centroid count: 500
Label count: 500
Schaefer - Number of coordinates: 500
Schaefer - MNI coordinates shape: (500, 3)
Schaefer - First 5 coordinates:
[[-36.3490566  -36.88207547 -22.19811321]
 [-28.63492063 -36.62222222 -15.08571429]
 [-34.98392283 -59.61414791 -17.10610932]
 [-25.21967213 -56.85245902 -10.01311475]
 [-37.5        -79.78658537 -15.68292683]]


In [45]:
# Combine coordinates (subcortical + cortical)
all_mni_coords = np.vstack([mni_coords_cortical, mni_coords_subcortical])  # (54 + 500, 3)

# 3. Create DataFrame
coords = pd.DataFrame({
    'name': all_labels,
    'x.mni': all_mni_coords[:, 0],
    'y.mni': all_mni_coords[:, 1],
    'z.mni': all_mni_coords[:, 2],
    'hemi': ['lh' if ('lh' in str(name)) or ('7Networks_LH_' in str(name)) else 
             'rh' if ('rh' in str(name)) or ('7Networks_RH_' in str(name)) else 
             'none' for name in all_labels],
    'type': ['cortical']*len(cortical_labels) + ['subcortical']*len(subcort_labels)
})

# Verify both modalities use the same label order
print(f"Total labels: {len(all_labels)}")

# Check first and last labels match
print(f"\nFirst label (should be cortical): {all_labels[0]}")
print(f"Last label (should be subcortical): {all_labels[-1]}")

# Check a few cortical-subcortical boundaries
print(f"\nLabel at index 499 (last cortical): {all_labels[499]}")
print(f"Label at index 500 (first subcortical): {all_labels[500]}")

# Verify coordinate counts
print(f"\nCoordinate count: {len(all_mni_coords)}")

Total labels: 554

First label (should be cortical): 7Networks_LH_Vis_1
Last label (should be subcortical): aGP-lh

Label at index 499 (last cortical): 7Networks_RH_Default_pCunPCC_11
Label at index 500 (first subcortical): HIP-head-m1-rh

Coordinate count: 554


In [ ]:
# Create coordinate DataFrame
coords = pd.DataFrame({
    'name': all_labels,
    'x.mni': all_mni_coords[:, 0],
    'y.mni': all_mni_coords[:, 1],
    'z.mni': all_mni_coords[:, 2],
    'type': ['cortical']*len(cortical_labels) + ['subcortical']*len(subcort_labels)
})

# Initialize 'hemi' and 'network'
coords['hemi'] = None
coords['network'] = None

# Cortical: extract 'hemi' and 'network'
cortical_mask = coords['type'] == 'cortical'
split_names_cort = coords.loc[cortical_mask, 'name'].str.split('_', expand=True)
coords.loc[cortical_mask, 'hemi'] = split_names_cort[1].str.lower()  # 'LH' → 'lh'
coords.loc[cortical_mask, 'network'] = split_names_cort[2]           # e.g. 'Vis'

# Subcortical: extract 'hemi' and assign 'network' as 'subcortical'
subcortical_mask = coords['type'] == 'subcortical'
coords.loc[subcortical_mask, 'hemi'] = coords.loc[subcortical_mask, 'name'].apply(
    lambda x: x.split('-')[-1] if x.endswith(('-lh', '-rh')) else 'none'
)
coords.loc[subcortical_mask, 'network'] = 'subcortical'
coords.to_csv('/UK_BB/brainbody/feature_imp/feature_imp_brain/mni_coords.csv', index=False) #dwmri/

In [ ]:
# Check coordinate-label alignment
print("="*60)
print("Coordinate-label alignment check")
print("="*60)

# 1. Check counts
print(f"\n1. Count verification:")
print(f"   Cortical labels: {len(cortical_labels)}")
print(f"   Subcortical labels: {len(subcort_labels)}")
print(f"   All labels: {len(all_labels)}")
print(f"   Cortical coords: {len(mni_coords_cortical)}")
print(f"   Subcortical coords: {len(mni_coords_subcortical)}")
print(f"   All coords: {len(all_mni_coords)}")

# 2. Check first few entries
print(f"\n2. First few entries verification:")

print(f"\n   First 5 CORTICAL labels:")
for i in range(min(5, len(cortical_labels))):
    print(f"      Label {i}: {cortical_labels[i]} → Coord: {mni_coords_cortical[i]}")

print(f"\n   First 5 SUBCORTICAL labels:")
for i in range(min(5, len(subcort_labels))):
    print(f"      Label {i}: {subcort_labels[i]} → Coord: {mni_coords_subcortical[i]}")

# 3. Check the combined order
print(f"\n3. Combined order verification:")

# Create a test DataFrame to visually verify
test_df = pd.DataFrame({
    'label': all_labels,
    'coord_index': range(len(all_labels)),
    'coord_x': all_mni_coords[:, 0],
    'coord_y': all_mni_coords[:, 1],
    'coord_z': all_mni_coords[:, 2],
    'type': ['cortical']*len(cortical_labels) + ['subcortical']*len(subcort_labels)
})

print(f"\n   First 5 combined (should be cortical):")
print(test_df.head())
print(f"\n   Last 5 combined (should be subcortical):")
print(test_df.tail())

# 4. Cross-check specific indices
print(f"\n4. Cross-check specific indices:")

# Check transition point (after cortical, before subcortical)
transition_idx = len(cortical_labels) - 1
print(f"\n   Last cortical (index {transition_idx}):")
print(f"      Label: {all_labels[transition_idx]}")
print(f"      Coord: {all_mni_coords[transition_idx]}")

print(f"\n   First subcortical (index {transition_idx + 1}):")
print(f"      Label: {all_labels[transition_idx + 1]}")
print(f"      Coord: {all_mni_coords[transition_idx + 1]}")

# 5. Verify by checking a known region
print(f"\n5. Known region check:")

# Pick a known cortical region (e.g., first Schaefer region)
print(f"\n   First Schaefer cortical region:")
print(f"      Label: {cortical_labels[0]}")
print(f"      Should be in all_labels[0]: {all_labels[0]}")
print(f"      Direct coord: {mni_coords_cortical[0]}")
print(f"      Combined coord: {all_mni_coords[0]}")
print(f"      Match? {np.allclose(mni_coords_cortical[0], all_mni_coords[0])}")

# Pick a known subcortical region (e.g., first Tian region)
print(f"\n   First Tian subcortical region:")
print(f"      Label: {subcort_labels[0]}")
subcort_start_idx = len(cortical_labels)
print(f"      Should be in all_labels[{subcort_start_idx}]: {all_labels[subcort_start_idx]}")
print(f"      Direct coord: {mni_coords_subcortical[0]}")
print(f"      Combined coord: {all_mni_coords[subcort_start_idx]}")
print(f"      Match? {np.allclose(mni_coords_subcortical[0], all_mni_coords[subcort_start_idx])}")

# 6. Sanity check: Are coordinates reasonable?
print(f"\n6. Coordinate sanity check:")
print(f"   Min X: {all_mni_coords[:, 0].min():.1f}, Max X: {all_mni_coords[:, 0].max():.1f}")
print(f"   Min Y: {all_mni_coords[:, 1].min():.1f}, Max Y: {all_mni_coords[:, 1].max():.1f}")
print(f"   Min Z: {all_mni_coords[:, 2].min():.1f}, Max Z: {all_mni_coords[:, 2].max():.1f}")
print(f"   Typical MNI range: X: ±100, Y: ±150, Z: ±100")

print("\n" + "="*60)
print("SUMMARY:")
print("="*60)

# Final verification
if len(all_labels) == len(all_mni_coords):
    print(f"✓ Counts match: {len(all_labels)} labels == {len(all_mni_coords)} coords")
else:
    print(f"✗ Counts DON'T match: {len(all_labels)} labels != {len(all_mni_coords)} coords")

# Check if cortical labels match the first part
first_cortical_match = all_labels[:len(cortical_labels)] == cortical_labels
print(f"✓ First {len(cortical_labels)} labels are cortical: {first_cortical_match}")

# Check if subcortical labels match the second part
if len(all_labels) >= len(cortical_labels) + len(subcort_labels):
    subcort_slice = all_labels[len(cortical_labels):len(cortical_labels)+len(subcort_labels)]
    subcortical_match = subcort_slice == subcort_labels
    print(f"✓ Next {len(subcort_labels)} labels are subcortical: {subcortical_match}")

Coordinate-label alignment check

1. Count verification:
   Cortical labels: 500
   Subcortical labels: 54
   All labels: 554
   Cortical coords: 500
   Subcortical coords: 54
   All coords: 554

2. First few entries verification:

   First 5 CORTICAL labels:
      Label 0: 7Networks_LH_Vis_1 → Coord: [-36.3490566  -36.88207547 -22.19811321]
      Label 1: 7Networks_LH_Vis_2 → Coord: [-28.63492063 -36.62222222 -15.08571429]
      Label 2: 7Networks_LH_Vis_3 → Coord: [-34.98392283 -59.61414791 -17.10610932]
      Label 3: 7Networks_LH_Vis_4 → Coord: [-25.21967213 -56.85245902 -10.01311475]
      Label 4: 7Networks_LH_Vis_5 → Coord: [-37.5        -79.78658537 -15.68292683]

   First 5 SUBCORTICAL labels:
      Label 0: HIP-head-m1-rh → Coord: [ 20. -12. -22.]
      Label 1: HIP-head-m2-rh → Coord: [ 22. -18. -16.]
      Label 2: THA-VAip-rh → Coord: [  8. -14.   4.]
      Label 3: THA-VAia-rh → Coord: [ 6. -6.  2.]
      Label 4: HIP-head-l-rh → Coord: [ 30. -14. -20.]

3. Combined order

In [ ]:
# Map indices to labels and save correlations with regions: dwMRI
n_regions = 554
expected_pairs = n_regions * (n_regions + 1) // 2
assert len(corr_dwi) == expected_pairs

# Generate all (i,j) pairs in upper triangle order (i <= j)
pairs = list(combinations_with_replacement(range(n_regions), 2))

# Add pair indices column
corr_dwi['node_pair'] = pairs
# Add flattened index column

corr_dwi['flattened_index'] = np.arange(len(corr_dwi))

# Add labeled connections
corr_dwi['connection'] = corr_dwi['node_pair'].apply(
    lambda pair: f"{all_labels[pair[0]]}-{all_labels[pair[1]]}"
)

# Split into separate columns
corr_dwi['region_i'] = corr_dwi['node_pair'].apply(lambda x: all_labels[x[0]])
corr_dwi['region_j'] = corr_dwi['node_pair'].apply(lambda x: all_labels[x[1]])
# For DWI: Add node_i and node_j columns to match rs-fMRI format
corr_dwi['node_i'] = corr_dwi['node_pair'].apply(lambda x: x[0])
corr_dwi['node_j'] = corr_dwi['node_pair'].apply(lambda x: x[1])

# Verify the results
print("First 5 connections:")
print(corr_dwi.head())
corr_dwi.to_csv('/UK_BB/brainbody/feature_imp/feature_imp_brain/dwmri/Schaefer7n200p_Tian_S1_Streamline_Count_i2_corr_with_g_stack_with_regions.csv', index=False)

First 5 connections:
   Feature  Pearson r  p-value  Valid_pairs  \
0        0     0.0141   0.0246        25346   
1        1    -0.0161   0.0104        25346   
2        2     0.0126   0.0446        25346   
3        3    -0.0415   0.0000        25346   
4        4    -0.0103   0.1012        25346   

                                     Modality node_pair  \
0  Schaefer7n200p_Tian_S1_Streamline_Count_i2    (0, 0)   
1  Schaefer7n200p_Tian_S1_Streamline_Count_i2    (0, 1)   
2  Schaefer7n200p_Tian_S1_Streamline_Count_i2    (0, 2)   
3  Schaefer7n200p_Tian_S1_Streamline_Count_i2    (0, 3)   
4  Schaefer7n200p_Tian_S1_Streamline_Count_i2    (0, 4)   

                              connection            region_i  \
0  7Networks_LH_Vis_1-7Networks_LH_Vis_1  7Networks_LH_Vis_1   
1  7Networks_LH_Vis_1-7Networks_LH_Vis_2  7Networks_LH_Vis_1   
2  7Networks_LH_Vis_1-7Networks_LH_Vis_3  7Networks_LH_Vis_1   
3  7Networks_LH_Vis_1-7Networks_LH_Vis_4  7Networks_LH_Vis_1   
4  7Networks_LH_Vis_1

In [ ]:
# Map indices to labels and save correlations with regions: rsMRI
# no diagonals, i < j ordering
n_regions = 554
expected_pairs = n_regions * (n_regions - 1) // 2  # 554*553/2 = 153,181

assert len(corr_rs) == expected_pairs, f"Expected {expected_pairs} pairs, got {len(corr_rs)}"

# Generate all (i,j) pairs in upper triangle order (i < j)
pairs = []
for i in range(n_regions):
    for j in range(i + 1, n_regions):  # i < j only
        pairs.append((i, j))

# Create mapping from feature index to (i,j) node pairs
feature_to_pair = {k: (i, j) for k, (i, j) in enumerate(pairs)}

# Add the node pair information to dataframe
corr_rs['node_i'] = corr_rs.index.map(lambda x: feature_to_pair[x][0])
corr_rs['node_j'] = corr_rs.index.map(lambda x: feature_to_pair[x][1])

# Add the label names
corr_rs['region_i'] = corr_rs['node_i'].map(lambda x: all_labels[x])
corr_rs['region_j'] = corr_rs['node_j'].map(lambda x: all_labels[x])

# Create the connection string
corr_rs['connection'] = corr_rs['region_i'] + '-' + corr_rs['region_j']
#  Add node_pair
corr_rs['node_pair'] = list(zip(corr_rs['node_i'], corr_rs['node_j']))
# Add flattened index column
corr_rs['flattened_index'] = np.arange(len(corr_rs))

# Now both have: node_pair, region_i, region_j, connection

# Verify the mapping
print("First 10 connections:")
for i in range(10):
    row = corr_rs.iloc[i]
    print(f"Index {i}: ({row['node_i']},{row['node_j']}) = {row['region_i']}-{row['region_j']} = {row['Pearson r']:.4f}")

print(f"\nLast connection:")
last_row = corr_rs.iloc[-1]
print(f"Index {len(corr_rs)-1}: ({last_row['node_i']},{last_row['node_j']}) = {last_row['region_i']}-{last_row['region_j']}")

# Quick check: what are the first and last node pairs?
print(f"\nFirst pair: (0,1) = {all_labels[0]}-{all_labels[1]}")
print(f"Last pair: (552,553) = {all_labels[552]}-{all_labels[553]}")
corr_rs.to_csv('/UK_BB/brainbody/feature_imp/feature_imp_brain/rsmri/Schaefer7n200p_Tian_S1_Full_Corr_i2_corr_with_g_stack_with_regions.csv', index=False)

First 10 connections:
Index 0: (0,1) = 7Networks_LH_Vis_1-7Networks_LH_Vis_2 = -0.0264
Index 1: (0,2) = 7Networks_LH_Vis_1-7Networks_LH_Vis_3 = -0.0413
Index 2: (0,3) = 7Networks_LH_Vis_1-7Networks_LH_Vis_4 = -0.0322
Index 3: (0,4) = 7Networks_LH_Vis_1-7Networks_LH_Vis_5 = -0.0428
Index 4: (0,5) = 7Networks_LH_Vis_1-7Networks_LH_Vis_6 = -0.0278
Index 5: (0,6) = 7Networks_LH_Vis_1-7Networks_LH_Vis_7 = 0.0077
Index 6: (0,7) = 7Networks_LH_Vis_1-7Networks_LH_Vis_8 = -0.0418
Index 7: (0,8) = 7Networks_LH_Vis_1-7Networks_LH_Vis_9 = -0.0307
Index 8: (0,9) = 7Networks_LH_Vis_1-7Networks_LH_Vis_10 = 0.0234
Index 9: (0,10) = 7Networks_LH_Vis_1-7Networks_LH_Vis_11 = 0.0164

Last connection:
Index 153180: (552,553) = pGP-lh-aGP-lh

First pair: (0,1) = 7Networks_LH_Vis_1-7Networks_LH_Vis_2
Last pair: (552,553) = pGP-lh-aGP-lh


In [31]:
# Save labeled correlation table as excel: dwMRI
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'dwmri_feature_imports_stack_labeled_sorted_{timestamp}.xlsx'), engine='openpyxl') as writer:
    corr_dwi.sort_values(by = 'Pearson r', ascending=False)[['region_i', 'region_j', 'Pearson r', 'p-value', 'node_pair', 'connection', 'flattened_index']].round(4).to_excel(writer, index=False)

with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'dwmri_feature_imports_stack_labeled_{timestamp}.xlsx'), engine='openpyxl') as writer:
    corr_dwi[['Pearson r', 'p-value', 'node_pair', 'connection', 'region_i', 'region_j']].round(4).to_excel(writer, index=False)

In [32]:
# Save labeled correlation table as excel: rsMRI
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'rsmri_feature_imports_stack_labeled_sorted_{timestamp}.xlsx'), engine='openpyxl') as writer:
    corr_rs.sort_values(by = 'Pearson r', ascending=False)[['region_i', 'region_j', 'Pearson r', 'p-value', 'node_pair', 'connection', 'flattened_index']].round(4).to_excel(writer, index=False)

with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'rsmri_feature_imports_stack_labeled_{timestamp}.xlsx'), engine='openpyxl') as writer:
    corr_rs[['Pearson r', 'p-value', 'node_pair', 'connection', 'region_i', 'region_j']].round(4).to_excel(writer, index=False)

## Prepare data for plotting

In [ ]:
# Functions to transform 1D correlations dataframe to 2D correlation matrix
def long_to_square_matrix_dwi(correlation, coordinates):
    regions = coordinates['name'].values
    
    # Initialize empty matrices
    n = len(regions)
    corr_matrix = np.zeros((n, n))
    pval_matrix = np.ones((n, n))  # Default p=1 for missing entries
    
    # Create name-to-index mapping
    name_to_idx = {name: idx for idx, name in enumerate(regions)}
    print(name_to_idx)
    # Fill matrices
    for _, row in correlation.iterrows():
        i = name_to_idx[row['region_i']]
        j = name_to_idx[row['region_j']]
        corr_matrix[i, j] = row['Pearson r']
        corr_matrix[j, i] = row['Pearson r']  # Symmetric
        pval_matrix[i, j] = row['p-value']
        pval_matrix[j, i] = row['p-value']    # Symmetric
    
    return (
        pd.DataFrame(corr_matrix, index=regions, columns=regions),
        pd.DataFrame(pval_matrix, index=regions, columns=regions)
    )

# -----------------
def long_to_square_matrix_rs(correlation, coordinates):
    """For functional connectome (no stored diagonals)"""
    regions = coordinates['name'].values
    n = len(regions)
    
    # Initialize matrices with NaN
    corr_matrix = np.full((n, n), np.nan)
    pval_matrix = np.full((n, n), np.nan)
    
    # Set diagonal to 1.0
    np.fill_diagonal(corr_matrix, 1.0)
    np.fill_diagonal(pval_matrix, 0.0)  # p=0 for self-correlation
    
    # Create name-to-index mapping
    name_to_idx = {name: idx for idx, name in enumerate(regions)}
    
    # Fill matrices from upper triangle data
    for _, row in correlation.iterrows():
        i = name_to_idx[row['region_i']]
        j = name_to_idx[row['region_j']]
        corr_matrix[i, j] = row['Pearson r']
        corr_matrix[j, i] = row['Pearson r']  # Mirror to lower triangle
        pval_matrix[i, j] = row['p-value']
        pval_matrix[j, i] = row['p-value']    # Mirror to lower triangle
    
    return (
        pd.DataFrame(corr_matrix, index=regions, columns=regions),
        pd.DataFrame(pval_matrix, index=regions, columns=regions)
    )

In [52]:
# A function to extract structure name and numeric index from cortical region names
def extract_structure_and_index(name):
    parts = name.split('_')
    if len(parts) >= 4:
        structure = '_'.join(parts[2:-1])  # e.g., 'Cont_pCun'
        try:
            index = int(parts[-1])
        except ValueError:
            index = 0
        return structure, index
    return '', 0

In [ ]:
# 0 Upload data
base_path = '/UK_BB/brainbody'
corr_dwi = pd.read_csv(os.path.join(base_path, f'/UK_BB/brainbody/feature_imp/feature_imp_brain/dwmri/Schaefer7n200p_Tian_S1_Streamline_Count_i2_corr_with_g_stack_with_regions.csv'))
corr_rs = pd.read_csv(os.path.join(base_path, f'/UK_BB/brainbody/feature_imp/feature_imp_brain/rsmri/Schaefer7n200p_Tian_S1_Full_Corr_i2_corr_with_g_stack_with_regions.csv'))
coords = pd.read_csv('/UK_BB/brainbody/feature_imp/feature_imp_brain/mni_coords.csv')

In [56]:
# 1 Assign 'Subcortical' as network for subcortical regions
coords['network'] = coords.apply(
    lambda row: row['network'] if row['type'] == 'cortical' else 'Subcortical',
    axis=1
)

# Assign hemisphere for subcortical regions based on name suffix
coords['hemi'] = coords.apply(
    lambda row: row['hemi'] if row['type'] == 'cortical' else (
        row['name'].split('-')[-1] if row['name'].endswith(('-lh', '-rh')) else 'none'
    ),
    axis=1
)

In [57]:
# 2 Convert 1D data frame to a correlation matrix 
corr_matrix_dwi, pval_matrix_dwi = long_to_square_matrix_dwi(corr_dwi, coords)
corr_matrix_rs, pval_matrix_rs = long_to_square_matrix_rs(corr_rs, coords)

{'7Networks_LH_Vis_1': 0, '7Networks_LH_Vis_2': 1, '7Networks_LH_Vis_3': 2, '7Networks_LH_Vis_4': 3, '7Networks_LH_Vis_5': 4, '7Networks_LH_Vis_6': 5, '7Networks_LH_Vis_7': 6, '7Networks_LH_Vis_8': 7, '7Networks_LH_Vis_9': 8, '7Networks_LH_Vis_10': 9, '7Networks_LH_Vis_11': 10, '7Networks_LH_Vis_12': 11, '7Networks_LH_Vis_13': 12, '7Networks_LH_Vis_14': 13, '7Networks_LH_Vis_15': 14, '7Networks_LH_Vis_16': 15, '7Networks_LH_Vis_17': 16, '7Networks_LH_Vis_18': 17, '7Networks_LH_Vis_19': 18, '7Networks_LH_Vis_20': 19, '7Networks_LH_Vis_21': 20, '7Networks_LH_Vis_22': 21, '7Networks_LH_Vis_23': 22, '7Networks_LH_Vis_24': 23, '7Networks_LH_Vis_25': 24, '7Networks_LH_Vis_26': 25, '7Networks_LH_Vis_27': 26, '7Networks_LH_Vis_28': 27, '7Networks_LH_Vis_29': 28, '7Networks_LH_Vis_30': 29, '7Networks_LH_Vis_31': 30, '7Networks_LH_Vis_32': 31, '7Networks_LH_Vis_33': 32, '7Networks_LH_Vis_34': 33, '7Networks_LH_Vis_35': 34, '7Networks_LH_Vis_36': 35, '7Networks_LH_Vis_37': 36, '7Networks_LH_Vis_3

In [ ]:
# 3 Incorporate coordinates to the corr table
# dwMRI
corr_with_coords_dwi = corr_dwi.merge(
    coords.rename(columns={'name':'region_i', 'x.mni':'x_i', 'y.mni':'y_i', 'z.mni':'z_i'}),
    on='region_i'
).merge(
    coords.rename(columns={'name':'region_j', 'x.mni':'x_j', 'y.mni':'y_j', 'z.mni':'z_j'}),
    on='region_j'
).drop(columns = ['hemi_x', 'type_x', 'network_x']).rename(columns={'hemi_y':'hemi', 'type_y':'type', 'network_y':'network'})

corr_with_coords_dwi.to_csv(os.path.join(base_path, 'feature_imp/feature_imp_brain/dwmri/corr_with_coords_dwi.csv'), index=False)

# rsMRI
corr_with_coords_rs = corr_rs.merge(
    coords.rename(columns={'name':'region_i', 'x.mni':'x_i', 'y.mni':'y_i', 'z.mni':'z_i'}),
    on='region_i'
).merge(
    coords.rename(columns={'name':'region_j', 'x.mni':'x_j', 'y.mni':'y_j', 'z.mni':'z_j'}),
    on='region_j'
).drop(columns = ['hemi_x', 'type_x', 'network_x']).rename(columns={'hemi_y':'hemi', 'type_y':'type', 'network_y':'network'})

corr_with_coords_rs.to_csv(os.path.join(base_path, 'feature_imp/feature_imp_brain/rsmri/corr_with_coords_rs.csv'), index=False)

In [ ]:
# 4 Reorder networkds
network_order = {
    'Vis': 0, 'SomMot': 1, 'DorsAttn': 2, 'SalVentAttn': 3,
    'Limbic': 4, 'Cont': 5, 'Default': 6, 'Subcortical': 7
}

# Apply to all regions
coords['structure'] = coords['name'].apply(
    lambda x: extract_structure_and_index(x)[0] if '7Networks' in x else x
)
coords['region_num'] = coords['name'].apply(
    lambda x: extract_structure_and_index(x)[1] if '7Networks' in x else 0
)

# Assign sorting ranks
coords['network_rank'] = coords['network'].map(network_order)
coords['hemi_rank'] = coords['hemi'].map({'lh': 0, 'rh': 1, 'none': 2})

# Sort by network, structure name, region number, then hemisphere
coords_sorted = coords.sort_values([
    'network_rank',
    'structure',
    'region_num',
    'hemi_rank'
])

# Final region order
final_order = coords_sorted['name'].tolist()

# Reorder coordinate dataframe accordingly
coords_aligned = (
    coords.set_index('name')     # or coords_sorted
          .loc[final_order]      # exact same order as the matrices
          .reset_index()         # get 'name' back as a column
)

# Save the reordered region names to a text file
with open('/UK_BB/brainbody/feature_imp/feature_imp_brain/conn_regions_reordered.txt', 'w') as f:
    f.write('\n'.join(final_order))

In [40]:
# 5 Reorder correlation matrix
corr_with_coords_dwi = pd.read_csv(os.path.join(base_path, 'feature_imp/feature_imp_brain/dwmri/corr_with_coords_dwi.csv'))
corr_with_coords_rs = pd.read_csv(os.path.join(base_path, 'feature_imp/feature_imp_brain/rsmri/corr_with_coords_rs.csv'))

# Read the region order file
with open(os.path.join(base_path, 'feature_imp/feature_imp_brain/conn_regions_reordered.txt'), 'r') as f:
    final_order = [line.strip() for line in f.readlines()]

# Verify the content
print(f"Number of regions: {len(final_order)}")
print("First 10 regions:")
for i, region in enumerate(final_order[:10]):
    print(f"{i}: {region}")

print("\nLast 10 regions:")
for i, region in enumerate(final_order[-10:], start=len(final_order)-10):
    print(f"{i}: {region}")

corr_matrix_ordered_dwi = corr_matrix_dwi.loc[final_order, final_order]
pval_matrix_ordered_dwi = pval_matrix_dwi.loc[final_order, final_order]

corr_matrix_ordered_rs = corr_matrix_rs.loc[final_order, final_order]
pval_matrix_ordered_rs = pval_matrix_rs.loc[final_order, final_order]

Number of regions: 554
First 10 regions:
0: 7Networks_LH_Vis_1
1: 7Networks_RH_Vis_1
2: 7Networks_LH_Vis_2
3: 7Networks_RH_Vis_2
4: 7Networks_LH_Vis_3
5: 7Networks_RH_Vis_3
6: 7Networks_LH_Vis_4
7: 7Networks_RH_Vis_4
8: 7Networks_LH_Vis_5
9: 7Networks_RH_Vis_5

Last 10 regions:
544: THA-VPm-lh
545: THA-VPm-rh
546: aGP-lh
547: aGP-rh
548: lAMY-lh
549: lAMY-rh
550: mAMY-lh
551: mAMY-rh
552: pGP-lh
553: pGP-rh


# sMRI

In [43]:
# sMRI
corr_smri = pd.read_csv(os.path.join(base_path, f'feature_imp/feature_imp_brain/struct_fs_aseg_volume_corr_with_g_stack.csv'))
print(corr_smri.columns.to_list())
print(corr_smri['Feature'])

['Feature', 'Pearson r', 'p-value', 'Valid_pairs', 'Modality']
0                 Volume of 3rd-Ventricle (whole brain)
1                 Volume of 4th-Ventricle (whole brain)
2                 Volume of 5th-Ventricle (whole brain)
3            Volume of Accumbens-area (left hemisphere)
4           Volume of Accumbens-area (right hemisphere)
5                  Volume of Amygdala (left hemisphere)
6                 Volume of Amygdala (right hemisphere)
7                    Volume of Brain-Stem (whole brain)
8                      Volume of BrainSeg (whole brain)
9               Volume of BrainSegNotVent (whole brain)
10          Volume of BrainSegNotVentSurf (whole brain)
11                  Volume of CC-Anterior (whole brain)
12                   Volume of CC-Central (whole brain)
13              Volume of CC-Mid-Anterior (whole brain)
14             Volume of CC-Mid-Posterior (whole brain)
15                 Volume of CC-Posterior (whole brain)
16                          Volume of CSF

# Additional: correlations between brain features and g predicted from *each* brain phenotype

In [ ]:
# Configuration
warnings.simplefilter(action='ignore', category=FutureWarning)
from datetime import datetime

folds = range(0, 5)
base_path = '/UK_BB/brainbody'
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

modalities_brain = [
    'Schaefer7n200p_Tian_S1_Streamline_Count_i2',
    'full_correlation_Schaefer7n500p_Tian_S4',
    'struct_fs_aseg_volume'
]
# Timestamp for output files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# Compute correlation between features and predicted g-factor for BRAIN modalities
modality_results = {}
unsorted_results = {}

# Process each brain modality
for modality in modalities_brain:
    print(f"\nProcessing modality: {modality}")
    features, g_pred = [], []
    
    for fold in folds:
        try:
            # Path to predicted g-factor values
            test_path = os.path.join(
                base_path, 
                'brain',
                'folds', 
                f'fold_{fold}', 
                'g_pred', 
                f'{modality}_g_pred_XGB_test_with_id_fold_{fold}.csv'
            )
            
            # Path to feature values
            features_path = os.path.join(
                base_path,
                'brain',
                'folds',
                f'fold_{fold}',
                'scaling',
                f'{modality}_test_deconf_fold_{fold}.csv'
            )

            # Check if files exist before reading
            if not os.path.exists(test_path):
                print(f"Warning: Test file not found for fold {fold} - {test_path}")
                continue
            if not os.path.exists(features_path):
                print(f"Warning: Features file not found for fold {fold} - {features_path}")
                continue

            # Read data
            g_pred_test = pd.read_csv(test_path)
            features_corrected = pd.read_csv(features_path)
            
            # Validate data shapes
            if g_pred_test.empty or features_corrected.empty:
                print(f"Warning: Empty data for fold {fold}")
                continue
            if len(g_pred_test) != len(features_corrected):
                print(f"Warning: Mismatched lengths for fold {fold} (g_pred: {len(g_pred_test)}, features: {len(features_corrected)})")
                continue
            
            features.append(features_corrected)
            g_pred.append(g_pred_test)
            
        except Exception as e:
            print(f"Error processing fold {fold} for {modality}: {str(e)}")
            continue
    
    # Skip modality if no valid data
    if not features or not g_pred:
        print(f"Skipping {modality} - no valid data")
        continue
    
    # Concatenate and validate
    features_concat = pd.concat(features, axis=0, ignore_index=True)
    g_pred_concat = pd.concat(g_pred, axis=0, ignore_index=True)
    
    # Additional validation after concatenation
    if features_concat.empty or g_pred_concat.empty:
        print(f"Skipping {modality} - empty data after concatenation")
        continue
    if len(features_concat) != len(g_pred_concat):
        print(f"Warning: Mismatched lengths after concatenation (features: {len(features_concat)}, g_pred: {len(g_pred_concat)})")
        # Proceed with caution
    
    correlations = []
    for feature in features_concat.columns:
        try:
            # Get data
            x = g_pred_concat[f'g_pred_test_{modality}'].values
            y = features_concat[feature].values
            
            # Create mask for non-NaN values
            mask = ~np.isnan(x) & ~np.isnan(y)
            valid_count = sum(mask)
            
            # Debug print for problematic features
            if valid_count < 10:
                print(f"Low valid pairs ({valid_count}) for feature: {feature}")
            
            # Skip if insufficient data
            if valid_count < 2:
                print(f"Skipping {feature} - insufficient data (n={valid_count})")
                correlations.append({
                    'Feature': feature,
                    'Pearson r': np.nan,
                    'p-value': np.nan,
                    'Valid_pairs': valid_count
                })
                continue
            
            # Check for zero variance
            if np.std(x[mask]) == 0 or np.std(y[mask]) == 0:
                print(f"Skipping {feature} - zero variance in x or y")
                correlations.append({
                    'Feature': feature,
                    'Pearson r': np.nan,
                    'p-value': np.nan,
                    'Valid_pairs': valid_count
                })
                continue
            
            # Calculate correlation
            r_pred, p_pred = stats.pearsonr(x[mask], y[mask])
            
            correlations.append({
                'Feature': feature,
                'Pearson r': r_pred,
                'p-value': p_pred,
                'Valid_pairs': valid_count
            })
            
        except Exception as e:
            print(f"Error calculating {feature} in {modality}: {str(e)}")
            correlations.append({
                'Feature': feature,
                'Pearson r': np.nan,
                'p-value': np.nan,
                'Valid_pairs': np.nan
            })
    
    # Create DataFrame
    df = pd.DataFrame(correlations)
    df['Modality'] = modality
    
    # Store results
    unsorted_results[modality] = df.copy()
    modality_results[modality] = df.sort_values('Pearson r', ascending=False, key=abs).reset_index(drop=True)
    
    # Save individual CSV
    os.makedirs(os.path.join(base_path, 'feature_imp', 'feature_imp_brain'), exist_ok=True)
    output_path = os.path.join(base_path, f'feature_imp/feature_imp_brain/{modality}_corr_with_g_pred.csv')
    df.round(4).to_csv(output_path, index=False)
    print(f"Saved CSV for {modality} with {len(df)} features")

# Save Excel files for brain modalities
try:
    with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'brain_modalities_unsorted_{timestamp}.xlsx'), 
                       engine='openpyxl') as writer:
        for modality, df in unsorted_results.items():
            try:
                sheet_name = modality[:31]
                df.round(4).to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error saving {modality} to unsorted Excel: {str(e)}")
                continue

    with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'brain_modalities_sorted_{timestamp}.xlsx'),
                       engine='openpyxl') as writer:
        # Individual sorted sheets
        for modality, df in modality_results.items():
            try:
                sheet_name = modality[:31]
                df.round(4).to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error saving {modality} to sorted Excel: {str(e)}")
                continue
        
        # Combined sorted sheet
        try:
            combined_sorted = pd.concat(
                [df.assign(Modality_Pretty=modality_names.get(modality, modality)) 
                 for modality, df in modality_results.items()],
                axis=0, ignore_index=True
            )
            combined_sorted.round(4).to_excel(writer, sheet_name='All_brain_sorted', index=False)
        except Exception as e:
            print(f"Error creating combined sheet: {str(e)}")

except Exception as e:
    print(f"Error creating Excel files: {str(e)}")

print("\nProcessing complete! Created:")
print(f"- Individual CSV files for each brain modality in 'feature_imp_brain' folder")
print(f"- brain_modalities_unsorted_{timestamp}.xlsx")
print(f"- brain_modalities_sorted_{timestamp}.xlsx (with 'All_brain_sorted' sheet)")

In [ ]:
# Timestamp for output files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Compute correlation between features and predicted g-factor for top brain phenotypes
modality_results = {}
unsorted_results = {}

# Process each brain modality
for modality in modalities_brain:
    print(f"\nProcessing modality: {modality}")
    features, g_pred = [], []
    
    for fold in folds:
        try:
            # Path to predicted g-factor values
            test_path = os.path.join(
                base_path, 
                'brain',
                'folds', 
                f'fold_{fold}', 
                'g_pred', 
                f'{modality}_g_pred_XGB_test_with_id_fold_{fold}.csv'
            )
            
            # Path to feature values
            features_path = os.path.join(
                base_path,
                'brain',
                'folds',
                f'fold_{fold}',
                'scaling',
                f'{modality}_test_deconf_fold_{fold}.csv'
            )

            # Check if files exist before reading
            if not os.path.exists(test_path):
                print(f"Warning: Test file not found for fold {fold} - {test_path}")
                continue
            if not os.path.exists(features_path):
                print(f"Warning: Features file not found for fold {fold} - {features_path}")
                continue

            # Read data
            g_pred_test = pd.read_csv(test_path)
            features_corrected = pd.read_csv(features_path)
            
            # Validate data shapes
            if g_pred_test.empty or features_corrected.empty:
                print(f"Warning: Empty data for fold {fold}")
                continue
            if len(g_pred_test) != len(features_corrected):
                print(f"Warning: Mismatched lengths for fold {fold} (g_pred: {len(g_pred_test)}, features: {len(features_corrected)})")
                continue
            
            features.append(features_corrected)
            g_pred.append(g_pred_test)
            
        except Exception as e:
            print(f"Error processing fold {fold} for {modality}: {str(e)}")
            continue
    
    # Skip modality if no valid data
    if not features or not g_pred:
        print(f"Skipping {modality} - no valid data")
        continue
    
    # Concatenate and validate
    features_concat = pd.concat(features, axis=0, ignore_index=True)
    g_pred_concat = pd.concat(g_pred, axis=0, ignore_index=True)
    
    # Additional validation after concatenation
    if features_concat.empty or g_pred_concat.empty:
        print(f"Skipping {modality} - empty data after concatenation")
        continue
    if len(features_concat) != len(g_pred_concat):
        print(f"Warning: Mismatched lengths after concatenation (features: {len(features_concat)}, g_pred: {len(g_pred_concat)})")
    
    correlations = []
    for feature in features_concat.columns:
        try:
            # Get data
            x = g_pred_concat[f'g_pred_test_{modality}'].values
            y = features_concat[feature].values
            
            # Create mask for non-NaN values
            mask = ~np.isnan(x) & ~np.isnan(y)
            valid_count = sum(mask)
            
            # Debug print for problematic features
            if valid_count < 10:
                print(f"Low valid pairs ({valid_count}) for feature: {feature}")
            
            # Skip if insufficient data
            if valid_count < 2:
                print(f"Skipping {feature} - insufficient data (n={valid_count})")
                correlations.append({
                    'Feature': feature,
                    'Pearson r': np.nan,
                    'p-value': np.nan,
                    'Valid_pairs': valid_count
                })
                continue
            
            # Check for zero variance
            if np.std(x[mask]) == 0 or np.std(y[mask]) == 0:
                print(f"Skipping {feature} - zero variance in x or y")
                correlations.append({
                    'Feature': feature,
                    'Pearson r': np.nan,
                    'p-value': np.nan,
                    'Valid_pairs': valid_count
                })
                continue
            
            # Calculate correlation
            r_pred, p_pred = stats.pearsonr(x[mask], y[mask])
            
            correlations.append({
                'Feature': feature,
                'Pearson r': r_pred,
                'p-value': p_pred,
                'Valid_pairs': valid_count
            })
            
        except Exception as e:
            print(f"Error calculating {feature} in {modality}: {str(e)}")
            correlations.append({
                'Feature': feature,
                'Pearson r': np.nan,
                'p-value': np.nan,
                'Valid_pairs': np.nan
            })
    
    # Create DataFrame
    df = pd.DataFrame(correlations)
    df['Modality'] = modality
    
    # Store results
    unsorted_results[modality] = df.copy()
    modality_results[modality] = df.sort_values('Pearson r', ascending=False, key=abs).reset_index(drop=True)
    
    # Save individual CSV
    os.makedirs(os.path.join(base_path, 'feature_imp', 'feature_imp_brain'), exist_ok=True)
    output_path = os.path.join(base_path, f'feature_imp/feature_imp_brain/{modality}_corr_with_g_pred.csv')
    df.round(4).to_csv(output_path, index=False)
    print(f"Saved CSV for {modality} with {len(df)} features")

# Save Excel files for brain modalities
try:
    with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'brain_modalities_unsorted_{timestamp}.xlsx'), 
                       engine='openpyxl') as writer:
        for modality, df in unsorted_results.items():
            try:
                sheet_name = modality[:31]
                df.round(4).to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error saving {modality} to unsorted Excel: {str(e)}")
                continue

    with pd.ExcelWriter(os.path.join(base_path, 'feature_imp/feature_imp_brain', f'brain_modalities_sorted_{timestamp}.xlsx'),
                       engine='openpyxl') as writer:
        # Individual sorted sheets
        for modality, df in modality_results.items():
            try:
                sheet_name = modality[:31]
                df.round(4).to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error saving {modality} to sorted Excel: {str(e)}")
                continue
        
        # Combined sorted sheet
        try:
            combined_sorted = pd.concat(
                [df.assign(Modality_Pretty=modality_names.get(modality, modality)) 
                 for modality, df in modality_results.items()],
                axis=0, ignore_index=True
            )
            combined_sorted.round(4).to_excel(writer, sheet_name='All_brain_sorted', index=False)
        except Exception as e:
            print(f"Error creating combined sheet: {str(e)}")

except Exception as e:
    print(f"Error creating Excel files: {str(e)}")

print("\nProcessing complete! Created:")
print(f"- Individual CSV files for each brain modality in 'feature_imp_brain' folder")
print(f"- brain_modalities_unsorted_{timestamp}.xlsx")
print(f"- brain_modalities_sorted_{timestamp}.xlsx (with 'All_brain_sorted' sheet)")